The goal of this notebook is to scrape this website: 

https://www.tran.sla.ny.gov/JSP/query/PublicQueryPremisesSearchPage.jsp

For all of the liquor licenses in NYC, and then clean them.

## NYC Zip code list


from: http://nyc.pediacities.com/New_York_City_ZIP_Codes

In [24]:
manhattan = [10001,10002,10003,10004,10005,
             10006,10007,10009,10010,10011,
             10012,10013,10014,10016,10017,
             10018,10019,10020,10021,10022,
             10023,10024,10025,10026,10027,
             10028,10029,10030,10031,10032,
             10033,10034,10035,10036,10037,
             10038,10039,10040,10044,10048,
             10065,10069,10075,10111,10115,
             10128,10280,10281,10282]

In [27]:
brooklyn = [11201,11203,11204,11205,11206,
            11207,11208,11209,11210,11211,
            11212,11213,11214,11215,11216,
            11217,11218,11219,11220,11221,
            11222,11223,11224,11225,11226,
            11228,11229,11230,11231,11232,
            11233,11234,11235,11236,11237,
            11238,11239,11251]

In [26]:
queens = [11001,11004,11005,11040,11096,
          11101,11102,11103,11104,11105,
          11106,11109,11354,11355,11356,
          11357,11358,11359,11360,11361,
          11362,11363,11364,11365,11366,
          11367,11368,11369,11371,11372,
          11373,11374,11375,11377,11378,
          11379,11385,11411,11412,11413,
          11414,11415,11416,11417,11418,
          11419,11420,11421,11422,11423,
          11426,11427,11428,11429,11430,
          11432,11433,11434,11435,11436,
          11451,11691,11692,11694,11697]

In [28]:
bronx = [10451,10452,10453,10454,10455,
         10456,10457,10458,10459,10460,
         10461,10462,10465,10466,10467,
         10468,10469,10470,10471,10472,
         10473,10474,10475]

In [29]:
staten_island = [10301,10302,10303,10304,10305,10306,10307,10308,10309,10310,10312,10314]

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import requests
from bs4 import BeautifulSoup
import time

In [6]:
def get_liquor_licenses(zipcode, start=1, stop=None):
    
    driver = webdriver.Firefox()
    driver.get("https://www.tran.sla.ny.gov/JSP/query/PublicQueryPremisesSearchPage.jsp")
    elem = driver.find_element_by_name("zipCode")
    elem.send_keys(zipcode)
    elem.send_keys(Keys.RETURN)
    
    #Get the result table
    result_table = driver.find_element_by_class_name("default")
    row_vals = result_table.find_elements_by_tag_name("tr")
    
    #Start the clock!
    start_time = time.time()
    
    # Loop over each entry in each row
    col_vals = []
    print "There are {} rows\n".format(len(row_vals))
    for idx, row in enumerate(row_vals[start:stop]):
        if idx % 10 == 0:
            print idx + start, round(time.time() - start_time, 3)
        if idx > 0 and idx % 100 == 0:
            print idx + start
            csv_name = '{}_{}_{}.csv'.format(zipcode, start, idx + start)
            with open(csv_name, 'wb') as f:
                writer = csv.writer(f)
                writer.writerows(col_vals)
            
            
        columns = row.find_elements_by_class_name("displayvalue")
        
        #Crawl out to the link in the first entry - it contains filing date and other info
        license_name = columns[0].find_element_by_css_selector('a')
        license_link = license_name.get_attribute("href")
        r = requests.get(license_link)
        soup = BeautifulSoup(r.text, 'html.parser')
        values = soup.findAll("td", { "class" : "displayvalue" })
        # labels unnecessary
        #labels = soup.findAll("td", { "class" : "displaylabel" })
        # Clean values and labels
        #link_features = zip([val.contents for val in labels if "\xa0" not in str(val)][0:7],
        #                    [val.contents for val in values if "\xa0" not in str(val)][0:7])
        
        column_text = [col.text for col in columns]
        link_text = [val.string for val in values if "\xa0" not in str(val)][0:7]
        
        col_vals.append(column_text + link_text)
        
    driver.close()
    
    return col_vals


In [18]:
zip_10001 = get_liquor_licenses(10001)

There are 1671 rows

0 0.0
10 12.845
20 22.402
30 32.265
40 41.677
50 52.266
60 116.263
70 126.604
80 136.127
90 146.369
100 156.101
100
110 165.811
120 175.916
130 185.737
140 208.759
150 382.202
160 393.104
170 404.537
180 414.294
190 424.988
200 434.804
200
210 493.227
220 503.154
230 513.358
240 524.563
250 534.879
260 618.862
270 646.093
280 657.98
290 667.955
300 677.548
300
310 687.737
320 838.766
330 849.223
340 859.975
350 900.413
360 1181.644
370 1191.539
380 1201.511
390 1211.496
400 1221.519
400
410 1231.571
420 1241.775
430 1251.591
440 1335.854
450 1911.384
460 2540.915
470 3193.84
480 3786.179
490 3939.742
500 3950.336
500
510 3960.986
520 3971.569
530 3981.977
540 3992.692
550 4004.12
560 4336.136
570 4974.986
580 5589.825
590 6207.351
600 7563.237
600


ConnectionError: HTTPSConnectionPool(host='www.tran.sla.ny.gov', port=443): Max retries exceeded with url: /servlet/ApplicationServlet?pageName=com.ibm.nysla.data.publicquery.PublicQuerySuccessfulResultsPage&validated=true&serialNumber=1121469&licenseType=OP (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x120c10c50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [3]:
zip_10001_b = get_liquor_licenses(10001,start=600)

There are 1673 rows

0 0.001
10 14.517
20 26.625
30 51.097
40 77.981
50 93.572
60 119.578
70 168.595
80 179.282
90 193.853
100 214.482
100
110 276.5
120 297.493
130 309.904
140 326.489
150 358.075
160 372.246
170 385.266
180 402.072


ConnectionError: HTTPSConnectionPool(host='www.tran.sla.ny.gov', port=443): Max retries exceeded with url: /servlet/ApplicationServlet?pageName=com.ibm.nysla.data.publicquery.PublicQuerySuccessfulResultsPage&validated=true&serialNumber=1152724&licenseType=OP (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x10dbda690>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [5]:
zip_10001_b = get_liquor_licenses(10001,start=700)

There are 1673 rows

0 0.0
10 14.32
20 35.347
30 47.824
40 60.873
50 72.869
60 118.153
70 130.778
80 142.603
90 253.754
100 659.315
100
110 1240.018
120 1307.265
130 1321.3
140 1332.228
150 1343.046
160 1812.186
170 2322.711
180 2500.209
190 2510.392
200 2520.662
200
210 2531.237
220 2541.364
230 2709.917
240 3205.481
250 3373.183
260 3389.289
270 3402.448
280 3415.152
290 3459.079
300 3472.771
300
310 3485.519
320 3496.385
330 3507.207
340 3935.995
350 3953.429
360 3971.213
370 3988.841
380 4391.189
390 4502.469
400 4521.973
400
410 4619.105
420 4746.211
430 4789.562
440 4817.966
450 4829.674
460 4842.894
470 4854.58
480 4892.964
490 4919.888
500 4945.8
500
510 4958.082
520 4969.625
530 4994.931
540 5040.046
550 5086.65
560 5131.689
570 5174.731
580 5274.673
590 5363.534
600 5376.934
600
610 5391.667
620 5438.164
630 5470.112
640 5486.765
650 5497.879
660 5509.635
670 5532.41
680 5579.979
690 5635.804
700 5678.821
700
710 5725.399
720 5767.912
730 5810.233
740 5928.009
750 5968.567
76

In [7]:
zip_10001_b

[[u'YK FRUIT & VEGETABLE CORP',
  u'383 8TH AVENUE\nW. 29TH & W.30TH STREETS\nNEW YORK, NY 10001',
  u'122',
  u'AX',
  u'08/31/2012',
  u'Expired',
  u'1141504',
  u'GROCERY BEER, WINE PROD',
  u'Expired',
  u'1',
  u'06/26/2003',
  u'09/01/2009',
  u'08/31/2012'],
 [u"MOON'S BROTHERS MARKET CORP.",
  u'383 8TH AVENUE\nNEW YORK, NY 10001',
  u'ML1',
  u'A',
  u'09/19/2003',
  u'License is Inactive',
  u'1141505',
  u'GROCERY STORE BEER',
  u'License is Inactive',
  u'1',
  u'06/26/2003',
  u'09/19/2003',
  u'09/19/2003'],
 [u'N & A PRODUCTIONS INC',
  u'27 W 35TH ST\nNEW YORK, NY 10001',
  u'901',
  u'OP',
  u'09/30/2017',
  u'License is Active',
  u'ON-PREMISES LIQUOR',
  u'License is Active',
  u'3',
  u'\n',
  u'N & A PRODUCTIONS INC',
  u'ADDL BAR 2ND FLOOR',
  u'1'],
 [u'IZUMI JAPANESE RESTAURANT INC',
  u'139 W 28TH ST STORE WEST\n6TH AVE & 7TH AVE\nNEW YORK, NY 10001',
  u'341',
  u'RW',
  u'07/31/2009',
  u'Expired',
  u'1141701',
  u'RESTAURANT WINE',
  u'Expired',
  u'1',
  